# Introduction

<bold><font size="5">Name: Fernaldy Aristo Wirjowerdojo</font></bold>

This programme was primarily created to perform the `Great Expectation` on the data after it has gone through the process of cleaning. Other tasks done within this notebook includes a local simulation of the workflow for the DAG (fetch, clean and post to elasticsearch) tasks.

# Data Background

This data was obtained from kaggle from [this link](https://www.kaggle.com/datasets/anshtanwar/global-data-on-sustainable-energy), it covers the progress of countries worldwide toward sustainable energy. Features include:
- **Entity**: The name of the country or region for which the data is reported.
- **Year**: The year for which the data is reported, ranging from 2000 to 2020.
- **Access to electricity (% of population)**: The percentage of population with access to electricity.
- **Access to clean fuels for cooking (% of population)**: The percentage of the population with primary reliance on clean fuels.
- **Renewable-electricity-generating-capacity-per-capita**: Installed Renewable energy capacity per person
- **Financial flows to developing countries (US $)**: Aid and assistance from developed countries for clean energy projects.
- **Renewable energy share in total final energy consumption (%)**: Percentage of renewable energy in final energy consumption.
- **Electricity from fossil fuels (TWh)**: Electricity generated from fossil fuels (coal, oil, gas) in terawatt-hours.
- **Electricity from nuclear (TWh)**: Electricity generated from nuclear power in terawatt-hours.
- **Electricity from renewables (TWh)**: Electricity generated from renewable sources (hydro, solar, wind, etc.) in terawatt-hours.
- **Low-carbon electricity (% electricity)**: Percentage of electricity from low-carbon sources (nuclear and renewables).
- **Primary energy consumption per capita (kWh/person)**: Energy consumption per person in kilowatt-hours.
- **Energy intensity level of primary energy (MJ/$2011 PPP GDP)**: Energy use per unit of GDP at purchasing power parity.
- **Value $CO_{2}$ emissions (metric tons per capita)**: Carbon dioxide emissions per person in metric tons.
- **Renewables (% equivalent primary energy)**: Equivalent primary energy that is derived from renewable sources.
- **GDP growth (annual %)**: Annual GDP growth rate based on constant local currency.
- **GDP per capita**: Gross domestic product per person.
- **Density ($P/Km^{2}$)**: Population density in persons per square kilometer.
- **Land Area ($Km^{2}$)**: Total land area in square kilometers.
- **Latitude**: Latitude of the country's centroid in decimal degrees.
- **Longitude**: Longitude of the country's centroid in decimal degrees.

---

# Import Library

In [1]:
import pandas as pd
import re
from sqlalchemy import create_engine
from sklearn.impute import KNNImputer
from elasticsearch import Elasticsearch, helpers
from great_expectations.data_context import FileDataContext

# Data Loading

In [4]:
def connect(username: str='postgres', password: str='postgres', host: str='postgres', port: int=5432, database: str='sdg', table_name: str='table_sdg'):
    """
    Creates a connection to the postgresql database given the arguments `username`, `password` and `database`. 
    Then, it grabs the table from the database following the argument `table_name` 
    Parameters:
    - username: Username to connect to the server
    - password: Password for the above username
    - host: Host connection endpoint
    - port: Port endpoint
    - database: Name of database to connect to
    - table_name: Table name inside the database to grab
    Returns the table in a pandas dataframe format

    Example usage:
    # connect('postgres', 'postgres', 'localhost', 5432, 'sales', 'my_table')
    -> Uses the username: `postgres` with the password: `postgres` to connect to the `sales` database
    -> Grabs the table called `my_table` from the `sales` database 
    """
    engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')
    table = pd.read_sql_table(table_name, engine)
    # table.to_csv('/opt/airflow/data_raw.csv', index=False) # For docker use
    return table # For notebook use

In [5]:
# Arguemnt for fetching data from DOCKER postgres database // Will not work if container is not running
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = 5434
database = 'sdg'
table_name = 'table_sdg'
df = connect(username, password, host, port, database, table_name)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3644 entries, 0 to 3643
Data columns (total 21 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   entity                                                           3644 non-null   object 
 1   Year                                                             3644 non-null   int64  
 2   Access to electricity (% of population)                          3634 non-null   float64
 3   Access to clean fuels for cooking                                3475 non-null   float64
 4   Renewable-electricity-generating-capacity-per-capita             2713 non-null   float64
 5   Financial flows to developing countries (US $)                   1555 non-null   float64
 6   Renewable energy share in the total final energy consumption (%  3450 non-null   float64
 7   Electricity from fossil fuels (TWh)       

In [7]:
df.sample(5)

,entity,Year,Access to electricity (% of population),Access to clean fuels for cooking,Renewable-electricity-generating-capacity-per-capita,Financial flows to developing countries (US $),Renewable energy share in the total final energy consumption (%,Electricity from fossil fuels (TWh),Electricity from nuclear (TWh),Electricity from renewables (TWh),...,Primary energy consumption per capita (kWh/person),Energy intensity level of primary energy (MJ/$2017 PPP GDP),value_co2_emissions_kt_by_country,Renewables (% equivalent primary energy),gdp_growth,gdp_per_capita,Densityn(P/Km2),Land Area(Km2),latitude,longitude
2896,Senegal,2009,53.50000,35.4,0.96,38090000.0,51.0,2.36,0.0,0.29,...,2122.7695,4.59,6670.0,NaN,2.752105,1308.93790,87.0,196722.0,14.497401,-14.452362
1179,Finland,2006,100.00000,100.0,NaN,NaN,31.0,36.68,22.0,22.46,...,72824.5550,6.41,67019.0,17.534887,4.027410,41188.09400,18.0,338145.0,61.924110,25.748152
2657,Portugal,2001,100.00000,100.0,NaN,NaN,20.0,30.43,0.0,15.74,...,28699.3850,3.26,61380.0,15.646869,1.943672,11724.61100,111.0,92212.0,39.399870,-8.224454
1913,Libya,2004,91.96264,NaN,0.12,NaN,2.0,18.99,0.0,0.00,...,39269.4400,7.11,48459.0,NaN,4.461630,5800.58900,4.0,1759540.0,26.335100,17.228330
2316,Nepal,2015,82.52396,29.0,30.61,18440000.0,85.0,0.00,0.0,3.45,...,979.0958,5.47,7190.0,NaN,3.976053,901.74963,203.0,147181.0,28.394857,84.124010


The column names need to be normalised by:
1. Applying lower case to all columns
2. Removing all the special characters: 
    - The column details (e.g. % of population) will be removed
    - The columns with hyphen `(-)` will be replaced with underscore `(_)`
3. Replace whitespace with underscore `(_)`

---

# Data Cleaning

In [8]:
df.duplicated().sum()

0

In [17]:
df.isnull().sum()

entity                                                                0
Year                                                                  0
Access to electricity (% of population)                              10
Access to clean fuels for cooking                                   169
Renewable-electricity-generating-capacity-per-capita                931
Financial flows to developing countries (US $)                     2089
Renewable energy share in the total final energy consumption (%     194
Electricity from fossil fuels (TWh)                                  21
Electricity from nuclear (TWh)                                      126
Electricity from renewables (TWh)                                    21
Low-carbon electricity (% electricity)                               42
Primary energy consumption per capita (kWh/person)                    0
Energy intensity level of primary energy (MJ/$2017 PPP GDP)         207
value_co2_emissions_kt_by_country                               

In [9]:
df.entity.nunique()

176

In [14]:
col_names = []
pct_missing = []
for col in df.columns:
    missing_pct = round(len(df[df[col].isnull()]) / len(df) * 100, 2)
    col_names.append(col)
    pct_missing.append(missing_pct)

missing_data = dict(zip(col_names, pct_missing))
missing_data = pd.DataFrame([missing_data]).T
missing_data.columns = ['Percentage of Missing Values']
missing_data.sort_values(by=missing_data.columns[0], ascending=False)

,Percentage of Missing Values
Renewables (% equivalent primary energy),58.59
Financial flows to developing countries (US $),57.33
Renewable-electricity-generating-capacity-per-capita,25.55
value_co2_emissions_kt_by_country,11.75
gdp_growth,8.70
gdp_per_capita,7.74
Energy intensity level of primary energy (MJ/$2017 PPP GDP),5.68
Renewable energy share in the total final energy consumption (%,5.32
Access to clean fuels for cooking,4.64
Electricity from nuclear (TWh),3.46


In [33]:
df[df['Financial flows to developing countries (US $)'].isna()].sample(15)

,entity,Year,Access to electricity (% of population),Access to clean fuels for cooking,Renewable-electricity-generating-capacity-per-capita,Financial flows to developing countries (US $),Renewable energy share in the total final energy consumption (%,Electricity from fossil fuels (TWh),Electricity from nuclear (TWh),Electricity from renewables (TWh),...,Primary energy consumption per capita (kWh/person),Energy intensity level of primary energy (MJ/$2017 PPP GDP),value_co2_emissions_kt_by_country,Renewables (% equivalent primary energy),gdp_growth,gdp_per_capita,Densityn(P/Km2),Land Area(Km2),latitude,longitude
398,Bermuda,2000,100.000000,NaN,NaN,NaN,0.0,0.00,0.0,0.00,...,39297.8800,1.52,517.0,NaN,9.317157,56284.1680,1281.0,54.0,32.299507,-64.790340
2738,Romania,2019,100.000000,87.7,NaN,NaN,23.0,23.27,11.0,24.63,...,19839.0470,2.38,73949.0,16.837948,4.188261,12899.3460,84.0,238391.0,45.943160,24.966760
3113,Spain,2015,100.000000,100.0,NaN,NaN,16.0,123.19,57.0,97.09,...,33920.2400,2.85,256279.0,17.507902,3.835173,25732.0180,94.0,505370.0,40.463670,-3.749220
3576,Vanuatu,2016,57.820000,8.8,33.43,NaN,33.0,0.05,0.0,0.02,...,2445.8684,3.71,180.0,NaN,4.688678,2805.6653,25.0,12189.0,-15.376706,166.959150
2793,Saint Lucia,2011,94.752520,95.4,0.11,NaN,12.0,0.36,0.0,0.00,...,10594.8580,2.99,NaN,NaN,NaN,NaN,301.0,616.0,13.909444,-60.978893
3307,Tonga,2020,100.000000,84.1,73.33,NaN,NaN,0.06,0.0,0.01,...,6542.0660,NaN,NaN,NaN,0.654699,4624.8230,147.0,747.0,-21.178986,-175.198240
361,Belize,2005,83.504776,82.9,134.99,NaN,31.0,0.10,0.0,0.07,...,9279.9450,5.21,469.0,NaN,1.940904,3885.0342,17.0,22966.0,17.189877,-88.497650
2055,Maldives,2020,100.000000,99.2,31.10,NaN,NaN,0.57,0.0,0.04,...,17308.5880,NaN,NaN,NaN,-33.499900,6924.1060,1802.0,298.0,3.202778,73.220680
2802,Saint Lucia,2020,100.000000,94.3,20.89,NaN,NaN,0.33,0.0,0.00,...,12810.4150,NaN,NaN,NaN,NaN,NaN,301.0,616.0,13.909444,-60.978893
3194,Sweden,2012,100.000000,100.0,NaN,NaN,48.0,4.09,64.0,98.30,...,68276.2100,4.59,42139.0,43.305737,-0.588305,58037.8200,25.0,450295.0,60.128162,18.643501


The missing values that occur on the column `Financial flows to developing countries (US $)` could be a mixture of missing at random and missing not at random because the missing values also occur on more developed countries. As such, dropping this column because there are 57% missing values would be more approriate.

In [55]:
df[df['Renewables (% equivalent primary energy)'].isna()].sample(15)

,entity,Year,Access to electricity (% of population),Access to clean fuels for cooking,Renewable-electricity-generating-capacity-per-capita,Financial flows to developing countries (US $),Renewable energy share in the total final energy consumption (%,Electricity from fossil fuels (TWh),Electricity from nuclear (TWh),Electricity from renewables (TWh),...,Primary energy consumption per capita (kWh/person),Energy intensity level of primary energy (MJ/$2017 PPP GDP),value_co2_emissions_kt_by_country,Renewables (% equivalent primary energy),gdp_growth,gdp_per_capita,Densityn(P/Km2),Land Area(Km2),latitude,longitude
1241,Gambia,2004,34.79930,3.6,0.26,NaN,61.0,0.17,0.0,0.00,...,898.55994,2.94,NaN,NaN,NaN,NaN,239.0,11300.0,13.466666,-16.566668
1252,Gambia,2015,54.56199,2.0,1.62,NaN,48.0,0.28,0.0,0.00,...,1028.85880,3.48,NaN,NaN,NaN,NaN,239.0,11300.0,13.466666,-16.566668
3227,Tajikistan,2003,100.00000,48.0,620.80,20780000.0,64.0,0.16,0.0,16.17,...,9982.50100,8.18,2180.0,NaN,10.999999,237.75734,68.0,144100.0,38.861034,71.276090
765,Comoros,2010,69.80771,3.2,2.10,NaN,66.0,0.04,0.0,0.00,...,1074.19650,2.78,170.0,NaN,3.799933,1316.49120,467.0,2235.0,-11.645500,43.333300
1353,Grenada,2011,88.80000,92.6,3.28,3090000.0,11.0,0.20,0.0,0.00,...,10196.20100,2.69,310.0,NaN,0.764977,7291.74170,331.0,349.0,12.116500,-61.679000
945,Dominica,2001,82.27921,82.0,110.23,NaN,9.0,0.05,0.0,0.03,...,8312.15800,2.63,150.0,NaN,-0.063845,4883.00300,96.0,751.0,15.414999,-61.370975
83,Angola,2020,46.89061,50.2,115.43,NaN,NaN,4.49,0.0,12.10,...,3527.16000,NaN,NaN,NaN,-5.399987,1776.16690,26.0,1246700.0,-11.202692,17.873886
22,Albania,2001,100.00000,40.5,NaN,NaN,NaN,0.13,0.0,3.52,...,8635.53200,3.89,3230.0,NaN,8.293313,1281.65980,105.0,28748.0,41.153330,20.168331
647,Cayman Islands,2018,100.00000,NaN,154.54,NaN,0.0,0.64,0.0,0.01,...,42507.16400,1.90,NaN,NaN,4.194341,86059.74000,274.0,264.0,19.329900,81.252400
2403,Nicaragua,2018,87.11668,53.8,104.20,19750000.0,51.0,2.05,0.0,2.48,...,4282.38700,4.46,5110.0,NaN,-3.363688,2014.57520,55.0,130370.0,12.865416,-85.207230


From a quick overview, the column `Renewable energy share in the total final energy consumption (%)` has a lot of missing values that only occurs on developing countries. To not introduce bias, similar to above, this column will also be dropped.

- Given the missing values, imputing values for the columns `Financial flows to developing countries (US $)` with 57% missing values and `Renewables (% equivalent primary energy)` with 58% missing values would introduce too much bias. 
> As such, dropping these two columns would be more appropriate. 
- The other columns with missing values will be imputed using `KNNImputer` because there are fixed columns such as latitude, longitude and density that are unique to each country. 
> So, `KNNImputer` would be the best imputer technique to use in this problem.
- Percentage columns `['access_to_electricity', 'access_to_clean_fuels_for_cooking', 'renewable_energy_share_in_the_total_final_energy_consumption', 'low_carbon_electricity']` except `'gdp_growth'` should max out at 100%
> The values greater than 100% will be capped out at 100%

In [105]:
def clean_data(data_path: str):
    """
    Given the argument `data_path` that is expected to be the file path to the raw data grabbed from the first function `connect`. 
    This function reads the data (.csv) into a pandas dataframe and performs data cleaning by:
    1. Normalising the column names -> Applying lowercase, removing details, whitespaces, non alpha-numeric characters except underscore
    2. Drops the column `financial_flows_to_developing_countries` and `renewables`
    3. Imputes the missing values using KNN Imputer 
    4. Creates a csv file of the cleaned and imputed data 

    # example usage:
    clean_data('/tmp/data_raw.csv')
    """
    data = pd.read_csv(data_path)
    
    ## Normalise columns
    normalised_columns = [] # List to store the normalised column names
    for col in data.columns:
        # Convert to lowercase
        col = col.lower()
    
        # Remove details in parentheses
        col = re.sub(r'\([^)]*\)', '', col)

        # Replace non-alphanumeric characters (except underscore) with space
        col = re.sub(r'[^a-z0-9_]', ' ', col)

        # Replace spaces and hyphens with underscore and strip trailing / leading underscores
        col = re.sub(r'[\s-]+', '_', col).strip('_')

        normalised_columns.append(col)
    data.columns = normalised_columns # Replace column names with the normalised column names

    # -----------------------------------------------------------------------------------------
    ## Drop columns: Renewables and financial flows
    data.drop(columns=['financial_flows_to_developing_countries', 'renewables'], axis=1, inplace=True)

    # -----------------------------------------------------------------------------------------
    ## Fixing percentage columns (except gdp growth) to max out at 100%
    pct_cols = ['access_to_electricity', 'access_to_clean_fuels_for_cooking', 'renewable_energy_share_in_the_total_final_energy_consumption', 'low_carbon_electricity']
    for col in pct_cols:
        data.loc[data[col] > 100, col] = 100
    
    # -----------------------------------------------------------------------------------------
    # Fix error in value input on density column
    # data['density_n'] = data['density_n'].str.replace(',', '.')
    # data['density_n'].astype('float')

    # -----------------------------------------------------------------------------------------
    ## Impute missing values
    # Separate numerical and categorical columns // list out the numerical column names 
    numerics, numeric_cols = data.select_dtypes(include='number'), data.select_dtypes(include='number').columns.to_list()
    categoricals = data.select_dtypes(include='object')

    # Create an instance of KNNImputer to impute the numerical columns
    imputer = KNNImputer(n_neighbors=15)
    numerics_imputed = imputer.fit_transform(numerics)
    numerics_df = pd.DataFrame(numerics_imputed, columns=numeric_cols, index=numerics.index)
    
    # Merge the imputed numerical columns with the categorical columns
    data_imputed = pd.concat([categoricals, numerics_df], axis=1)

    # Imputing the final missing value due to conversion error
    # data_imputed.fillna(data['density_n'].median(), inplace=True)

    # -----------------------------------------------------------------------------------------
    ## Fixing year data type
    data_imputed['year'] = data_imputed['year'].astype('int')

    # -----------------------------------------------------------------------------------------
    ## Create a csv file of the cleaned data to save locally
    data_imputed.to_csv('data_clean.csv', index=False)
    
    ## Return the cleaned data for notebook
    return data_imputed

NOTE: From the function above, there is a small section on fixing the error that occurred on the `density` column and `fillna` method near the end due to the conversion error that happened when importing the raw data into the postgres database. The raw data has the column `Density` is of datatype `float` but has a single input using `','` for decimal separator. As such, postgres interprets this column as `VARCHAR` while pandas can interpret the column as float. 

In [106]:
df_clean = clean_data('data_raw.csv')

In [52]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3644 entries, 0 to 3643
Data columns (total 19 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   entity                                                        3644 non-null   object 
 1   year                                                          3644 non-null   int64  
 2   access_to_electricity                                         3644 non-null   float64
 3   access_to_clean_fuels_for_cooking                             3644 non-null   float64
 4   renewable_electricity_generating_capacity_per_capita          3644 non-null   float64
 5   renewable_energy_share_in_the_total_final_energy_consumption  3644 non-null   float64
 6   electricity_from_fossil_fuels                                 3644 non-null   float64
 7   electricity_from_nuclear                                      3644 no

In [20]:
df_clean.isnull().sum()

entity                                                          0
year                                                            0
access_to_electricity                                           0
access_to_clean_fuels_for_cooking                               0
renewable_electricity_generating_capacity_per_capita            0
renewable_energy_share_in_the_total_final_energy_consumption    0
electricity_from_fossil_fuels                                   0
electricity_from_nuclear                                        0
electricity_from_renewables                                     0
low_carbon_electricity                                          0
primary_energy_consumption_per_capita                           0
energy_intensity_level_of_primary_energy                        0
value_co2_emissions_kt_by_country                               0
gdp_growth                                                      0
gdp_per_capita                                                  0
densityn  

---

# Data Export

In [ ]:
def post_to_elasticsearch(data_path: str, index_name: str, es_host='http://elasticsearch:9200'):
    """
    Given the arguments: `data_path`, `index_name` and `es_host`, this function fetches the cleaned data to:
    1. Convert to json format (dictionary)
    2. Creates the elasticsearch client
    3. Posts the data into elasticsearch
    """
    ## Read the csv file and convert to json format
    data = pd.read_csv(data_path)

    # ----------------------------------------------
    ## Create the Elasticsearch client
    es = Elasticsearch(hosts=es_host)

    # ----------------------------------------------
    ## Prepare data for bulk indexing
    actions = [
        {
            "_index": index_name,
            "_source": row.to_dict()
        }
        for _, row in data.iterrows()
    ]
    helpers.bulk(es, actions)

---

# Great Expectations

## Setup

In [2]:
context = FileDataContext.create(project_root_dir='./')

In [3]:
# Datasource
datasource = context.sources.add_pandas('great_expectation_sustainable_energy')

# Data asset
asset_name = 'sustainable_energy'
path_to_data = 'data_clean.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()

In [4]:
# Expectation suite
expectation_suite_name = 'expectation-sustainable-energy'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,entity,year,access_to_electricity,access_to_clean_fuels_for_cooking,renewable_electricity_generating_capacity_per_capita,renewable_energy_share_in_the_total_final_energy_consumption,electricity_from_fossil_fuels,electricity_from_nuclear,electricity_from_renewables,low_carbon_electricity,primary_energy_consumption_per_capita,energy_intensity_level_of_primary_energy,value_co2_emissions_kt_by_country,gdp_growth,gdp_per_capita,densityn,land_area,latitude,longitude
0,Afghanistan,2000,1.613591,6.2,9.22,44.0,0.16,0.0,0.31,65.957440,302.59482,1.64,760.0,5.560600,405.536571,60.0,652230.0,33.93911,67.70995
1,Afghanistan,2001,4.074574,7.2,8.86,45.0,0.09,0.0,0.50,84.745766,236.89185,1.74,730.0,5.560600,405.536571,60.0,652230.0,33.93911,67.70995
2,Afghanistan,2002,9.409158,8.2,8.47,37.0,0.13,0.0,0.56,81.159424,210.86215,1.40,1029.0,5.560600,179.426570,60.0,652230.0,33.93911,67.70995
3,Afghanistan,2003,14.738506,9.5,8.09,36.0,0.31,0.0,0.63,67.021280,229.96822,1.40,1220.0,8.832278,190.683800,60.0,652230.0,33.93911,67.70995
4,Afghanistan,2004,20.064968,10.9,7.75,44.0,0.33,0.0,0.56,62.921350,204.23125,1.20,1029.0,1.414118,211.382080,60.0,652230.0,33.93911,67.70995


## Expectations

In [2]:
gx_raw_df = pd.read_csv('data_raw.csv')
gx_df = pd.read_csv('data_clean.csv')
df_clean_columns = gx_df.columns

In [8]:
# Expectation 1: renewable electricity was most imputed so it should not be null
validator.expect_column_values_to_not_be_null('renewable_electricity_generating_capacity_per_capita')

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 3644,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [10]:
# Expectation 2: Percentage columns should be between 0 and 100 // 
pct_cols = ['access_to_electricity', 'access_to_clean_fuels_for_cooking', 'renewable_energy_share_in_the_total_final_energy_consumption', 'low_carbon_electricity']
cols_tested = len(pct_cols)
success_count = 0
for col in pct_cols:
    pct_valid = validator.expect_column_values_to_be_between(column=col, min_value=0, max_value=100)
    if pct_valid['success'] == True:
        success_count += 1
if success_count == cols_tested:
    print(f'Amount of columns tested: {cols_tested}', '\n',
          f'Success count: {success_count}')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Amount of columns tested: 4 
 Success count: 4


In [11]:
# Expectation 3: GDP per capita should be of type float //
float_cols = [col for col in gx_df.columns if col not in ['entity', 'year']]
cols_tested = len(float_cols)
success_count = 0
for col in float_cols:
    float_valid = validator.expect_column_values_to_be_of_type(column=col, type_='float')
    if float_valid['success'] == True:
        success_count += 1
if success_count == cols_tested:
    print(f'Amount of columns tested: {cols_tested}', '\n',
          f'Success count: {success_count}')


Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Amount of columns tested: 17 
 Success count: 17


In [12]:
# Expectation 4: Entity column of the cleaned data should still have the unique values as the raw data //
set_entity = gx_raw_df['entity'].unique()
validator.expect_column_values_to_be_in_set(column='entity', value_set=set_entity)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 3644,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [13]:
# Since this is a time-series data and each country has around 21 rows for each year, the 'to be unique' expectation can not be met. 
# Instead the expectation 'expect compound columns to be unique' will be used 
# Expectation 5: Entity and year should be unique when paired together // 
validator.expect_compound_columns_to_be_unique(column_list=['entity', 'year'])

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 3644,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [14]:
# Expectation 6: GDP per capita should have a mean within considerable amounts
validator.expect_column_mean_to_be_between(column='gdp_per_capita', min_value=1000, max_value=100000)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 13249.121110618735
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [15]:
# Expectation 7: Entity or country names should not exceed the longest country name: The United Kingdom of Great Britain and Northern Ireland and shortest: Chad
validator.expect_column_value_lengths_to_be_between(column='entity', min_value=4, max_value=56)

Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 3644,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [18]:
# Expectation 8: Latitude and longitude should be of type float or int
cols_tested = 2
success_count = 0
for col in ['latitude', 'longitude']:
    valid = validator.expect_column_values_to_be_in_type_list(column=col, type_list=['float', 'int'])
    if valid['success'] == True:
        success_count += 1
if success_count == cols_tested:
    print(f'Amount of columns tested: {cols_tested}', '\n',
          f'Success count: {success_count}')

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Amount of columns tested: 2 
 Success count: 2


In [19]:
# Save expectation suite
validator.save_expectation_suite(discard_failed_expectations=False)

## Checkpoint

In [20]:
# Checkpoint
checkpoint = context.add_or_update_checkpoint(
    name='checkpoint',
    validator=validator
)

In [21]:
checkpoint_result = checkpoint.run()

Calculating Metrics:   0%|          | 0/56 [00:00<?, ?it/s]

## Data Docs

In [ ]:
context.build_data_docs()

---